### -Open with Salesforce Environment-

In [85]:
import pandas as pd
from datetime import date, datetime, timezone, timedelta
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [2]:
username = 'adriel.piacquadio@engbim.com'
password = 'Adriel2021'
security_token = 'UiFCCsvyGhC115hMDBpeeC6K'

sf = Salesforce(username=username, password=password, security_token=security_token)
print(sf)

### Query for Opportunities

In [3]:
def query_salesforce_opps(sf, date):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Project__c, Id, Name, CreatedDate, Proposal_Delivery_Date__c, Internal_Owner__c, Amount, Deal_Type__c, CloseDate, Estimator__c, IsWon, Division__c, Trades__c, Project_stage__c, End_Market__c, Point_of_Contact_within_Account__c, Customer_Classification__c, StageName FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate > {date}"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

In [4]:
def query_salesforce_opps2(sf, date1, date2):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Project__c, Id, Name, CreatedDate, Proposal_Delivery_Date__c, Internal_Owner__c, Amount, Deal_Type__c, CloseDate, Estimator__c, IsWon, Division__c, Trades__c, Project_stage__c, End_Market__c, Point_of_Contact_within_Account__c, Customer_Classification__c, StageName FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate > {date1} AND  CreatedDate < {date2}"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

df_opps_00 = query_salesforce_opps2(sf, '2021-09-08T00:41:46.000+0000', '2022-01-17T12:47:33.000+0000')

In [5]:
# Fecha inicial para la consulta
date_opps = '2021-01-01T00:00:00.000+0000'
all_df_opps = []

while True:
    # Obtener los datos desde Salesforce
    df_opps = query_salesforce_opps(sf, date_opps)
    
    # Si no hay más datos, salir del bucle
    if df_opps.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_opps.append(df_opps)
    
    # Actualizar la fecha para la siguiente consulta
    date_opps = df_opps['CreatedDate'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_opps) < 2000:
        break

In [6]:
# Combinar todos los DataFrames en uno solo
df_opps_combined = pd.concat(all_df_opps, ignore_index=True)

# Combinar df_opps_00 con df_opps_combined
df_opps_final = pd.concat([df_opps_00, df_opps_combined], ignore_index=True)
df_opps_final = df_opps_final.drop_duplicates()

In [7]:
#df_opps.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Opportunities.csv', index=False)

### Query for Projects

In [8]:
def query_salesforce_proj(sf, id):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Id, Name, Project_State__c FROM Project__c WHERE Id >= '{id}'"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

In [9]:
# Id inicial para la consulta
id_proj = 'a0p7V000007JxehQAC'
all_df_proj = []

while True:
    # Obtener los datos desde Salesforce
    df_proj = query_salesforce_proj(sf, id_proj)
    
    # Si no hay más datos, salir del bucle
    if df_proj.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_proj.append(df_proj)
    
    # Actualizar el Id para la siguiente consulta
    id_proj = df_proj['Id'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_proj) < 2000:
        break

In [10]:
df_proj_final = pd.concat(all_df_proj, ignore_index=True)
df_proj_final = df_proj_final.drop_duplicates()

In [11]:
#df_proj.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Projects.csv', index=False)

### Query for Clients & Contacts

In [12]:
def query_salesforce_acc(sf, id):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT AccountId, Title, Name, Id, Account.Industry, Account.Name, Account.Sales_Division__c, Account.Type, Account.Type__c, Account.BillingAddress FROM Contact WHERE Id >= '{id}' ORDER BY Id ASC"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)

    # Extraer datos anidados en la columna 'Account' y convertirlos en columnas separadas
    account_data = df['Account'].apply(lambda x: pd.Series(x, dtype='object'))  # Esto convierte diccionarios en columnas
    account_data.columns = ['Account.' + col for col in account_data.columns]  # Prefijar con 'Account.'
    if 'Account.attributes' in account_data.columns:
        account_data.drop(columns=['Account.attributes'], inplace=True)
    
    # Concatenar los datos de Account al DataFrame principal
    df = pd.concat([df.drop(columns=['Account']), account_data], axis=1)
    
    return df

In [13]:
# Id inicial para la consulta
id_acc = '0030e00002GntyVAAR'
all_df_acc = []

while True:
    # Obtener los datos desde Salesforce
    df_acc = query_salesforce_acc(sf, id_acc)
    
    # Si no hay más datos, salir del bucle
    if df_acc.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_acc.append(df_acc)
    
    # Actualizar el Id para la siguiente consulta
    id_acc = df_acc['Id'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_acc) < 2000:
        break

df_acc_combined = pd.concat(all_df_acc, ignore_index=True)

In [14]:
# Get not accesible IDs

query_acc_00 = f"SELECT AccountId, Title, Name, Id, Account.Industry, Account.Name, Account.Sales_Division__c, Account.Type, Account.Type__c, Account.BillingAddress FROM Contact WHERE Id >= '0030P00001t16W8QAI' AND Id <= '0030P00002HHH52QAH' ORDER BY Id ASC"
response_acc_00 = sf.query(query_acc_00)

# Extraer registros y crear DataFrame
records_acc_00 = response_acc_00['records']
df_acc_00 = pd.DataFrame(records_acc_00)

# Eliminar la columna 'attributes'
df_acc_00.drop(columns=['attributes'], inplace=True)

# Extraer datos anidados en la columna 'Account' y convertirlos en columnas separadas
account_data_00 = df_acc_00['Account'].apply(lambda x: pd.Series(x, dtype='object'))  # Esto convierte diccionarios en columnas
account_data_00.columns = ['Account.' + col for col in account_data_00.columns]  # Prefijar con 'Account.'
account_data_00.drop(columns=['Account.attributes'], inplace=True)

# Concatenar los datos de Account al DataFrame principal
df_acc_00 = pd.concat([df_acc_00.drop(columns=['Account']), account_data_00], axis=1)

In [15]:
# Combinar df_acc_00 con df_acc_combined
df_acc_final = pd.concat([df_acc_00, df_acc_combined], ignore_index=True)

# Función para obtener el valor de 'state' de un OrderedDict
def get_state_value(x):
    if isinstance(x, OrderedDict):
        return x.get('state', None)
    else:
        return None

# Aplicar la función a la columna 'Account.BillingAddress'
df_acc_final['Account.BillingAddress'] = df_acc_final['Account.BillingAddress'].apply(get_state_value)

df_acc_final = df_acc_final.drop_duplicates()

In [16]:
#df_acc.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\ContactsAccounts.csv', index=False)

### Joining SOQL Objects: Project - Opp. - Account - Contact

In [17]:
#df_opps = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Opportunities.csv')
#df_acc = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\ContactsAccounts.csv')
#df_proj = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Projects.csv')

In [18]:
df_opps_final = df_opps_final.rename(columns = {'Project__c':'ProjectId', 'Id':'OppId', 'Name':'Opportunity', 'Proposal_Delivery_Date__c':'DeliveryDate',
                                    'Internal_Owner__c':'InternalOwner', 'Sales_Division__c':'SalesDivision', 
                                    'Deal_Type__c':'DealType', 'Estimator__c':'Estimator', 'Division__c':'Division',
                                    'Trades__c':'Trades', 'Project_stage__c':'ProjectStage', 'End_Market__c':'EndMarket',
                                    'Point_of_Contact_within_Account__c':'ContactId', 'Customer_Classification__c':'CustomerClass'})

In [19]:
print(df_opps_final.columns)

Index(['ProjectId', 'OppId', 'Opportunity', 'CreatedDate', 'DeliveryDate',
       'InternalOwner', 'Amount', 'DealType', 'CloseDate', 'Estimator',
       'IsWon', 'Division', 'Trades', 'ProjectStage', 'EndMarket', 'ContactId',
       'CustomerClass', 'StageName'],
      dtype='object')


In [20]:
df_opps_final = df_opps_final[['ProjectId', 'OppId','Opportunity', 'Division', 'DealType', 'CreatedDate',
                               'DeliveryDate', 'CloseDate', 'InternalOwner', 'Estimator', 'Trades',
                               'EndMarket', 'ProjectStage', 'StageName','IsWon', 'Amount',
                               'CustomerClass', 'ContactId']]

In [21]:
df_proj_final = df_proj_final.rename(columns = {'Id':'ProjectId', 'Name':'Project', 'Project_State__c':'ProjectState'})

In [22]:
df_acc_final = df_acc_final.rename(columns = {'Account.Industry':'AccountIndustry', 'Account.Name':'AccountName',
                                              'Account.Sales_Division__c':'SalesDivision', 'Account.Type':'AccountType',
                                              'Account.Type__c':'CompanyType', 'Id':'ContactId',
                                              'Account.BillingAddress':'BillingAddress'})

In [23]:
df_acc_final = df_acc_final[['AccountId', 'AccountName', 'BillingAddress','AccountType',
                             'AccountIndustry', 'CompanyType', 'SalesDivision', 
                             'Name', 'Title', 'ContactId']]

In [24]:
df_opps_proj = pd.merge(df_opps_final, df_proj_final, on='ProjectId', how='left')

In [25]:
df_opps_proj.fillna({'Project': df_opps_proj['Opportunity']}, inplace=True)

In [26]:
df = pd.merge(df_opps_proj, df_acc_final, on='ContactId', how='left')

In [27]:
print(df.columns)

Index(['ProjectId', 'OppId', 'Opportunity', 'Division', 'DealType',
       'CreatedDate', 'DeliveryDate', 'CloseDate', 'InternalOwner',
       'Estimator', 'Trades', 'EndMarket', 'ProjectStage', 'StageName',
       'IsWon', 'Amount', 'CustomerClass', 'ContactId', 'Project',
       'ProjectState', 'AccountId', 'AccountName', 'BillingAddress',
       'AccountType', 'AccountIndustry', 'CompanyType', 'SalesDivision',
       'Name', 'Title'],
      dtype='object')


In [28]:
df = df[['ProjectId', 'Project', 'OppId', 'Opportunity', 'ProjectState', 'Division', 'DealType',
         'CreatedDate', 'DeliveryDate', 'CloseDate', 'InternalOwner', 'Estimator',
         'Trades', 'EndMarket', 'ProjectStage', 'StageName','IsWon', 'Amount', 'ContactId',
         'AccountId', 'AccountName', 'BillingAddress','AccountType', 'CustomerClass',
         'AccountIndustry', 'CompanyType', 'SalesDivision',
         'Name', 'Title']]

In [29]:
#df.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test.csv', index=False)

### Predicting Conversion Chances

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import r2_score
import lightgbm as lgb
from scipy.stats import uniform, randint
import pytz

In [31]:
regex_opp_1 = r'CO(-|([0-9]{1,2}))'
mask_1 = df['DealType'].isna() | (df['DealType'] == '')
mask_regex_1 = df['Opportunity'].str.contains(regex_opp_1, na=False)
df.loc[mask_1 & mask_regex_1, 'DealType'] = 'Closed Budget – Change Order'

regex_opp_2 = r'T&M'
mask_regex_2 = df['Opportunity'].str.contains(regex_opp_2, na=False)
df.loc[mask_1 & mask_regex_2, 'DealType'] = 'T&M New Contract (with NTE)'

df['DealType'] = df['DealType'].fillna('Closed Budget – New Contract')

C:\Users\AdrielPiacquadioENG\AppData\Local\Temp\ipykernel_27208\947589054.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_regex_1 = df['Opportunity'].str.contains(regex_opp_1, na=False)


In [32]:
df['CreatedDate'] = pd.to_datetime(df['CreatedDate'])
df['CreatedDate2'] = df['CreatedDate'].dt.strftime('%Y-%m-%d')

In [33]:
df = df.dropna(subset=['ContactId'])

In [34]:
df['ProjectState'] = np.where(df['ProjectState'].isnull(), df['BillingAddress'], df['ProjectState'])
df['ProjectState'] = df['ProjectState'].fillna('N/A')

In [35]:
columns = ['InternalOwner', 'SalesDivision', 'Estimator', 'Trades', 'ProjectStage', 'IsWon', 'ContactId', 'Name', 'AccountId', 'AccountName', 'AccountType', 'CustomerClass', 'ProjectState']

# Verificar si alguna de las columnas tiene valores nulos
columnas_con_nulos = df[columns].isnull().any()

# Filtrar el DataFrame para eliminar las filas con valores nulos en esas columnas
if not columnas_con_nulos.empty:
    df = df.dropna(subset=columnas_con_nulos.index)

In [36]:
# Obtener una lista de nombres únicos en la columna 'Division'
unique_divisions = df['Division'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
division_to_id = {division: i+1 for i, division in enumerate(unique_divisions)}

# Crear la nueva columna 'DivisionId' utilizando el diccionario de mapeo
df['DivisionId'] = df['Division'].map(division_to_id)

In [37]:
# Obtener una lista de nombres únicos en la columna 'DealType'
unique_deals = df['DealType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
deal_to_id = {deal: i+1 for i, deal in enumerate(unique_deals)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['DealTypeId'] = df['DealType'].map(deal_to_id)

In [38]:
# Obtener una lista de nombres únicos en la columna 'InternalOwner'
unique_owners = df['InternalOwner'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
owner_to_id = {owner: i+1 for i, owner in enumerate(unique_owners)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['InternalOwnerId'] = df['InternalOwner'].map(owner_to_id)

In [39]:
df['SalesDivisionId'] = np.where(df['SalesDivision'] == 'Inbound', 1, 
                                 np.where(df['SalesDivision'] == 'Outbound', 2, 3))

In [40]:
# Obtener una lista de nombres únicos en la columna 'Estimator'
unique_estimators = df['Estimator'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
estimator_to_id = {estimator: i+1 for i, estimator in enumerate(unique_estimators)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['EstimatorId'] = df['Estimator'].map(estimator_to_id)

In [41]:
# Obtener una lista de nombres únicos en la columna 'Trades'
unique_trades = df['Trades'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
trade_to_id = {trade: i+1 for i, trade in enumerate(unique_trades)}

# Crear la nueva columna 'TradeId' utilizando el diccionario de mapeo
df['TradeId'] = df['Trades'].map(trade_to_id)

In [42]:
# Obtener una lista de nombres únicos en la columna 'EndMarket'
unique_endmarkets = df['EndMarket'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
endmarket_to_id = {endmarket: i+1 for i, endmarket in enumerate(unique_endmarkets)}

# Crear la nueva columna 'EndMarketId' utilizando el diccionario de mapeo
df['EndMarketId'] = df['EndMarket'].map(endmarket_to_id)

In [43]:
df['ProjectStageId'] = np.where(df['ProjectStage'] == 'Bidding', 1, 
                                 np.where(df['ProjectStage'] == 'Awarded', 2, 3))

In [44]:
# Obtener una lista de nombres únicos en la columna 'ContactId'
unique_contacts = df['ContactId'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
contact_to_id = {contact: i+1 for i, contact in enumerate(unique_contacts)}

# Crear la nueva columna 'ContactId_int' utilizando el diccionario de mapeo
df['ContactId_int'] = df['ContactId'].map(contact_to_id)

In [45]:
# Obtener una lista de nombres únicos en la columna 'AccountId'
unique_accounts = df['AccountId'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
account_to_id = {account: i+1 for i, account in enumerate(unique_accounts)}

# Crear la nueva columna 'AccountId_int' utilizando el diccionario de mapeo
df['AccountId_int'] = df['AccountId'].map(account_to_id)

In [46]:
# Obtener una lista de nombres únicos en la columna 'AccountType'
unique_acctypes = df['AccountType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
acctype_to_id = {acctype: i+1 for i, acctype in enumerate(unique_acctypes)}

# Crear la nueva columna 'AccountTypeId' utilizando el diccionario de mapeo
df['AccountTypeId'] = df['AccountType'].map(acctype_to_id)

In [47]:
# Obtener una lista de nombres únicos en la columna 'CustomerClass'
unique_custclass = df['CustomerClass'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
custclass_to_id = {custclass: i+1 for i, custclass in enumerate(unique_custclass)}

# Crear la nueva columna 'CustomerClassId' utilizando el diccionario de mapeo
df['CustomerClassId'] = df['CustomerClass'].map(custclass_to_id)

In [48]:
# Obtener una lista de nombres únicos en la columna 'AccountIndustry'
unique_accindustry = df['AccountIndustry'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
accindustry_to_id = {accindustry: i+1 for i, accindustry in enumerate(unique_accindustry)}

# Crear la nueva columna 'AccountIndustryId' utilizando el diccionario de mapeo
df['AccountIndustryId'] = df['AccountIndustry'].map(accindustry_to_id)

In [49]:
# Obtener una lista de nombres únicos en la columna 'CompanyType'
unique_companytype = df['CompanyType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
companytype_to_id = {companytype: i+1 for i, companytype in enumerate(unique_companytype)}

# Crear la nueva columna 'CompanyTypeId' utilizando el diccionario de mapeo
df['CompanyTypeId'] = df['CompanyType'].map(companytype_to_id)

In [50]:
# Obtener una lista de nombres únicos en la columna 'ProjectState'
unique_projstate = df['ProjectState'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
projstate_to_id = {projstate: i+1 for i, projstate in enumerate(unique_projstate)}

# Crear la nueva columna 'CustomerClassId' utilizando el diccionario de mapeo
df['ProjectStateId'] = df['ProjectState'].map(projstate_to_id)

In [51]:
df['IsWonId'] = 0  # Inicialmente, establecemos todos los valores en 0
df.loc[df['IsWon'] == True, 'IsWonId'] = 1  # Si IsWon es True, asignamos 1 a IsWonId

In [52]:
df = df[['ProjectId', 'Project', 'OppId','Opportunity', 'ProjectState', 'ProjectStateId', 'Division',
         'DivisionId', 'DealType', 'DealTypeId','CreatedDate', 'CreatedDate2', 'DeliveryDate', 'CloseDate',
         'InternalOwner', 'InternalOwnerId', 'SalesDivision', 'SalesDivisionId',
         'Estimator', 'EstimatorId', 'Trades', 'TradeId', 'EndMarket', 'EndMarketId',
         'ProjectStage', 'ProjectStageId', 'StageName', 'IsWon', 'IsWonId',
         'Amount', 'AccountId', 'AccountName', 'AccountId_int', 'AccountType',
         'AccountTypeId', 'CustomerClass', 'CustomerClassId', 'AccountIndustry', 'AccountIndustryId',
         'CompanyType', 'CompanyTypeId', 'ContactId', 'Name', 'ContactId_int', 'Title']]

In [53]:
df1 = df.copy()

timezone = pytz.timezone('America/Los_Angeles')
df1['CreatedDate'] = pd.to_datetime(df1['CreatedDate'])
date_df_predictions = (datetime.now() - relativedelta(months=6)).astimezone(timezone)
df1 = df1.loc[(df1['CreatedDate'] < date_df_predictions) | (df1['IsWon'] == True)]

df1.drop(columns=['ProjectId', 'OppId', 'Division', 'DeliveryDate', 'CloseDate',
                  'EndMarket', 'StageName', 'AccountIndustry', 'CompanyType', 'Title'], inplace=True)

In [54]:
print(df1.columns)

Index(['Project', 'Opportunity', 'ProjectState', 'ProjectStateId',
       'DivisionId', 'DealType', 'DealTypeId', 'CreatedDate', 'CreatedDate2',
       'InternalOwner', 'InternalOwnerId', 'SalesDivision', 'SalesDivisionId',
       'Estimator', 'EstimatorId', 'Trades', 'TradeId', 'EndMarketId',
       'ProjectStage', 'ProjectStageId', 'IsWon', 'IsWonId', 'Amount',
       'AccountId', 'AccountName', 'AccountId_int', 'AccountType',
       'AccountTypeId', 'CustomerClass', 'CustomerClassId',
       'AccountIndustryId', 'CompanyTypeId', 'ContactId', 'Name',
       'ContactId_int'],
      dtype='object')


In [55]:
# Seleccionar las características y la variable objetivo
X = df1[['ProjectStateId', 'DealTypeId', 'InternalOwnerId',
         'SalesDivisionId', 'EstimatorId', 'TradeId',
         'ProjectStageId', 'ContactId_int', 'AccountId_int',
         'AccountTypeId', 'CustomerClassId', 'EndMarketId',
         'CompanyTypeId', 'DivisionId', 'AccountIndustryId']]
y = df1['IsWonId']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los parámetros que queremos probar
param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'n_estimators': randint(50, 200),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_split_gain': uniform(0, 0.3),
    'reg_alpha': uniform(0, 0.5),
    'reg_lambda': uniform(0, 0.5)
}

# Inicializar el modelo
lgb_model = lgb.LGBMClassifier(random_state=42)

# Configurar la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator=lgb_model, param_distributions=param_dist,
                                   n_iter=50, scoring='roc_auc', n_jobs=-1, cv=5, random_state=42, verbose=2)

# Ejecutar la búsqueda aleatoria
random_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Best parameters found: ", random_search.best_params_)
print("Best AUC-ROC Score: ", random_search.best_score_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de prueba
best_model = random_search.best_estimator_
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_pred_proba_best)

print(f'Optimized LightGBM - Accuracy: {accuracy_best * 100:.2f}%')
print(f'Optimized LightGBM - Precision: {precision_best:.2f}')
print(f'Optimized LightGBM - Recall: {recall_best:.2f}')
print(f'Optimized LightGBM - AUC-ROC: {roc_auc_best:.2f}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 2614, number of negative: 3588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000741 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 862
[LightGBM] [Info] Number of data points in the train set: 6202, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.421477 -> initscore=-0.316713
[LightGBM] [Info] Start training from score -0.316713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [56]:
# Obtener los valores únicos de las columnas 'InternalOwner', 'SalesDivision', etc.
unique_states = df['ProjectState'].unique()
unique_deals = df['DealType'].unique()
unique_internal_owners = df['InternalOwner'].unique()
unique_sales_divisions = df['SalesDivision'].unique()
unique_estimators = df['Estimator'].unique()
unique_trades = df['Trades'].unique()
unique_projstages = df['ProjectStage'].unique()
unique_contacts = df['Name'].unique()
unique_accounts = df['AccountName'].unique()
unique_acctypes = df['AccountType'].unique()
unique_custclasses = df['CustomerClass'].unique()
unique_endmarkets = df['EndMarket'].unique()
unique_comptypes = df['CompanyType'].unique()
unique_divisions = df['Division'].unique()
unique_accindustries = df['AccountIndustry'].unique()

# Crea un diccionario para mapear valores a IDs
state_id_map = {state: state_id for state_id, state in enumerate(unique_states, start=1)}
deal_id_map = {deal: deal_id for deal_id, deal in enumerate(unique_deals, start=1)}
internal_owner_id_map = {owner: owner_id for owner_id, owner in enumerate(unique_internal_owners, start=1)}
sales_division_id_map = {division: division_id for division_id, division in enumerate(unique_sales_divisions, start=1)}
estimator_id_map = {estimator: estimator_id for estimator_id, estimator in enumerate(unique_estimators, start=1)}
trade_id_map = {trade: trade_id for trade_id, trade in enumerate(unique_trades, start=1)}
projstage_id_map = {projstage: projstage_id for projstage_id, projstage in enumerate(unique_projstages, start=1)}
contact_id_map = {contact: contact_id for contact_id, contact in enumerate(unique_contacts, start=1)}
account_id_map = {account: account_id for account_id, account in enumerate(unique_accounts, start=1)}
acctype_id_map = {acctype: acctype_id for acctype_id, acctype in enumerate(unique_acctypes, start=1)}
custclass_id_map = {custclass: custclass_id for custclass_id, custclass in enumerate(unique_custclasses, start=1)}
endmarket_id_map = {endmarket: endmarket_id for endmarket_id, endmarket in enumerate(unique_endmarkets, start=1)}
comptype_id_map = {comptype: comptype_id for comptype_id, comptype in enumerate(unique_comptypes, start=1)}
division_id_map = {division: division_id for division_id, division in enumerate(unique_divisions, start=1)}
accindustry_id_map = {accindustry: accindustry_id for accindustry_id, accindustry in enumerate(unique_accindustries, start=1)}

In [57]:
#df2.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test3.csv', index=False)

In [58]:
#date_test = '2024-06-01T00:00:00.000+0000'

#query_test = f"SELECT Id, Name, CreatedDate, Internal_Owner__c, IsWon, Conversion_Chances__c FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate >= {date_test}"
#response_test = sf.query(query_test)

#records_test = response_test['records']
#df_test = pd.DataFrame(records_test)

#df_test.drop(columns=['attributes'], inplace=True)

In [59]:
#df_test.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test4.csv', index=False)

In [60]:
#opportunity_id = '006Uy0000095iCvIAI'
#probability_value = 79.63

#sf.Opportunity.update(opportunity_id, {'Conversion_Chances__c': probability_value})

### Predicting Closing Date

In [70]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import catboost as cb
from catboost import CatBoostRegressor

In [71]:
df3 = df.copy()

In [72]:
df3 = df3.sort_values(by='CreatedDate')
df3 = df3.loc[df3['IsWon'] == True]

In [73]:
df3['DeliveryDate'] = df3['DeliveryDate'].fillna(df3['CreatedDate2'])

In [74]:
df3[['Opportunity', 'CreatedDate', 'CreatedDate2', 'DeliveryDate', 'CloseDate', 'IsWon']].head(5)

,Opportunity,CreatedDate,CreatedDate2,DeliveryDate,CloseDate,IsWon
721,21-BKI-Alcon JCA 4S - As-built,2021-01-04 15:25:37+00:00,2021-01-04,2021-01-04,2021-01-12,True
723,C0782-PPSC-21-Golden West College Language Arts,2021-01-04 16:21:01+00:00,2021-01-04,2021-01-04,2021-03-31,True
726,20-TAC-UTRGV ION CO04,2021-01-04 18:44:00+00:00,2021-01-04,2021-01-04,2020-12-30,True
729,Northside Gwinnett ED Expansion CO01,2021-01-04 21:18:24+00:00,2021-01-04,2021-01-04,2021-01-04,True
743,20-LOT-Dell Childrens Hospital-CO04,2021-01-05 21:03:02+00:00,2021-01-05,2021-01-05,2021-04-20,True


In [75]:
len(df3)

3264

In [76]:
df3['CreatedDate2'] = pd.to_datetime(df3['CreatedDate2'], format='%Y-%m-%d')
df3['DeliveryDate'] = pd.to_datetime(df3['DeliveryDate'], format='%Y-%m-%d')
df3['CloseDate'] = pd.to_datetime(df3['CloseDate'], errors='coerce')  # Asume formato ISO por defecto, ajusta si es necesario

df3['DeliveryDate'] = df3['DeliveryDate'].fillna(df3['CreatedDate2'])

In [77]:
df3['ClosingTime'] = np.where(df3['CloseDate'].notnull(), 
                                (df3['CloseDate'] - df3['CreatedDate2']).dt.days, 
                                 np.nan)

In [78]:
df3['ClosingTime'] = np.where(df3['ClosingTime'] < 0, 0, df3['ClosingTime'])

In [79]:
df3 = df3.sort_values(by='ClosingTime')

In [80]:
# Seleccionar las características y la variable objetivo

X_df3 = df3[['ProjectStateId', 'DivisionId', 'DealTypeId', 'InternalOwnerId',
         'SalesDivisionId', 'EstimatorId', 'TradeId', 'EndMarketId',
         'ProjectStageId', 'ContactId_int', 'AccountId_int',
         'AccountTypeId', 'CustomerClassId', 'AccountIndustryId',
         'CompanyTypeId']]
y_df3 = df3['ClosingTime']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_df3, X_test_df3, y_train_df3, y_test_df3 = train_test_split(X_df3, y_df3, test_size=0.2, random_state=42)

# Definir el modelo CatBoost
cat_model_df3 = CatBoostRegressor(loss_function='MAE', logging_level='Silent')

# Definir la cuadrícula de hiperparámetros
param_grid_df3 = {
    'iterations': np.arange(100, 1001, 100),
    'depth': np.arange(2, 11, 2),
    'learning_rate': np.linspace(0.01, 0.2, 20),
    'l2_leaf_reg': np.logspace(-3, 1, 5),
    'border_count': np.arange(32, 129, 32)
}

# Búsqueda aleatoria de hiperparámetros
random_search_df3 = RandomizedSearchCV(
    estimator=cat_model_df3,
    param_distributions=param_grid_df3,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1
)

# Ajustar el modelo con los mejores hiperparámetros
random_search_df3.fit(X_train_df3, y_train_df3)

# Obtener los mejores hiperparámetros
best_params_df3 = random_search_df3.best_params_

# Ajustar el modelo con los mejores hiperparámetros
cat_model_tuned_df3 = CatBoostRegressor(loss_function='MAE', **best_params_df3, logging_level='Silent')
cat_model_tuned_df3.fit(X_train_df3, y_train_df3)

# Hacer predicciones con el modelo ajustado
y_pred_tuned_df3 = cat_model_tuned_df3.predict(X_test_df3)

# Evaluar el rendimiento del modelo ajustado
mae_tuned_df3 = mean_absolute_error(y_test_df3, y_pred_tuned_df3)
mse_tuned_df3 = mean_squared_error(y_test_df3, y_pred_tuned_df3)
rmse_tuned_df3 = np.sqrt(mse_tuned_df3)
r2_tuned_df3 = r2_score(y_test_df3, y_pred_tuned_df3)

# Imprimir las métricas del modelo ajustado
print(f'Modelo CatBoost con hiperparámetros optimizados:')
print(f'Mean Absolute Error (MAE): {mae_tuned_df3:.2f}')
print(f'Mean Squared Error (MSE): {mse_tuned_df3:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse_tuned_df3:.2f}')
print(f'Coefficient of Determination (R²): {r2_tuned_df3:.2f}')

Modelo CatBoost con hiperparámetros optimizados:
Mean Absolute Error (MAE): 15.89
Mean Squared Error (MSE): 2066.77
Root Mean Squared Error (RMSE): 45.46
Coefficient of Determination (R²): 0.14


In [90]:
fecha_actual = datetime.now()
fecha_antesdeayer = fecha_actual - timedelta(days=2)
fecha_antesdeayer_str = fecha_antesdeayer.strftime('%Y-%m-%d')

In [91]:
print(fecha_antesdeayer_str)

2024-06-23


In [86]:
# Función para obtener el ID mapeado
def get_mapped_id(value, id_map):
    return id_map.get(value)

def predecir_iswon(project_state, deal_type, internal_owner,
                   sales_division, estimator, trade,
                   project_stage, contact_id, account_id,
                   account_type, customer_class, end_market,
                   company_type, division, account_industry):
    # Mapear valores a IDs
    state_id = get_mapped_id(project_state, state_id_map)
    deal_id = get_mapped_id(deal_type, deal_id_map)
    internal_owner_id = get_mapped_id(internal_owner, internal_owner_id_map)
    sales_division_id = get_mapped_id(sales_division, sales_division_id_map)
    estimator_id = get_mapped_id(estimator, estimator_id_map)
    trade_id = get_mapped_id(trade, trade_id_map)
    projstage_id = get_mapped_id(project_stage, projstage_id_map)
    contact_id = get_mapped_id(contact_id, contact_id_map)
    account_id = get_mapped_id(account_id, account_id_map)
    acctype_id = get_mapped_id(account_type, acctype_id_map)
    custclass_id = get_mapped_id(customer_class, custclass_id_map)
    endmarket_id = get_mapped_id(end_market, endmarket_id_map)
    comptype_id = get_mapped_id(company_type, comptype_id_map)
    division_id = get_mapped_id(division, division_id_map)
    accindustry_id = get_mapped_id(account_industry, accindustry_id_map)
    # Haz lo mismo para las otras columnas

    # Crear un DataFrame con los datos de entrada
    data = pd.DataFrame({
        'ProjectStateId': [state_id],
        'DealTypeId': [deal_id],
        'InternalOwnerId': [internal_owner_id],
        'SalesDivisionId': [sales_division_id],
        'EstimatorId': [estimator_id],
        'TradeId': [trade_id],
        'ProjectStageId': [projstage_id],
        'ContactId_int': [contact_id],
        'AccountId_int': [account_id],
        'AccountTypeId': [acctype_id],
        'CustomerClassId': [custclass_id],
        'EndMarketId': [endmarket_id],
        'CompanyTypeId': [comptype_id],
        'DivisionId': [division_id],
        'AccountIndustryId': [accindustry_id]
    })

    # Realizar la predicción con el modelo
    prediction = best_model.predict(data)
    
    # Realizar la predicción de probabilidades con el modelo
    predicted_proba = best_model.predict_proba(data)
    
    # Realizar la predicción de Closing Time
    predicted_closing = cat_model_tuned_df3.predict(data)
    predicted_closing_rounded = int(round(predicted_closing[0], 0))
    predicted_closing_date = fecha_actual + timedelta(days=predicted_closing_rounded)
    predicted_closing_date_str = predicted_closing_date.strftime('%Y-%m-%d')
    
    # Obtener la probabilidad de que la predicción sea igual a 1
    prob_iswon = predicted_proba[:, best_model.classes_ == 1]
    
    # Devolver la predicción y la probabilidad resultante
    return prediction[0], prob_iswon[0][0] * 100, predicted_closing_date_str # Predicción y probabilidad en porcentaje

In [92]:
df4 = df.copy()
df4 = df4.loc[(df4['CreatedDate'] >= fecha_antesdeayer_str)]

if len(df4) > 0:
    # Obtener las predicciones y probabilidades de conversión para cada proyecto en df4
    def obtener_probabilidad(row):
        close_date_str = row['CloseDate']
        if row['IsWon']:
            return 1, 100, close_date_str
        else:
            chances, probabilidad, closing = predecir_iswon(row['ProjectState'], row['DealType'],
                                                   row['InternalOwner'], row['SalesDivision'],
                                                   row['Estimator'], row['Trades'],
                                                   row['ProjectStage'], row['Name'],
                                                   row['AccountName'], row['AccountType'],
                                                   row['CustomerClass'], row['EndMarket'],
                                                   row['CompanyType'], row['Division'],
                                                   row['AccountIndustry'])
            return chances, probabilidad, closing

    df4['WillBeWon?'], df4['ConversionChances'], df4['ExpectedClosing'] = zip(*df4.apply(obtener_probabilidad, axis=1))
    
else:
    print("No projects found for today's date")

In [88]:
for index, row in df4.iterrows():
    opportunity_id = row['OppId']
    probability_value = row['ConversionChances']
    expected_closing = row['ExpectedClosing']
    
    # Obtener los detalles actuales de la oportunidad
    opportunity_details = sf.Opportunity.get(opportunity_id)
    
    # Verificar el valor del campo 'Are_you_using_a_package_strategy__c'
    package_strategy = opportunity_details.get('Are_you_using_a_package_strategy__c', 'No')
    
    # Si el campo está vacío, asignar 'No'
    if not package_strategy:
        package_strategy = 'No'
    
    # Actualizar la oportunidad con los nuevos valores
    sf.Opportunity.update(opportunity_id, {
        'Conversion_Chances__c': probability_value,
        'ExpectedClosing__c': expected_closing,
        'Are_you_using_a_package_strategy__c': package_strategy
    })